In [1]:
import json
import pandas as pd
import requests
import paho.mqtt.client as mqtt

def load_json_url(url):
    response = requests.get(url)
    raw = response.json()
    data = [v for v in raw.values() if isinstance(v, dict)]
    df = pd.DataFrame(data)
    df["waktu"] = df["waktu"].str.replace(".", ":", regex=False)

    # Parse datetime
    df["timestamp"] = pd.to_datetime(df["waktu"], format="%d/%m/%Y, %H:%M:%S")

    # Sort
    df = df.sort_values("timestamp").reset_index(drop=True)

    return df

# Firebase Realtime Database perlu akhiran .json
url = "https://iotexperiment-9b165-default-rtdb.asia-southeast1.firebasedatabase.app/sensor/temperature.json"

df = load_json_url(url)
print(df.head())



   ldr  pir                waktu           timestamp
0  164  0.0  6/12/2025, 00:15:27 2025-12-06 00:15:27
1  164  0.0  6/12/2025, 00:15:27 2025-12-06 00:15:27
2  630  1.0  6/12/2025, 00:15:29 2025-12-06 00:15:29
3  630  1.0  6/12/2025, 00:15:29 2025-12-06 00:15:29
4  553  0.0  6/12/2025, 00:15:31 2025-12-06 00:15:31


In [2]:
# ============================================================
# 2. AUTO-LABELING (SAMA SEPERTI SEBELUMNYA)
# ============================================================
def label_activity(row):
    pir = row["pir"]
    ldr = row["ldr"]

    if pir == 0 and ldr > 500:
        return "ruangan kosong"
    if pir == 0 and ldr < 200:
        return "orang baru keluar"
    if pir == 1 and ldr < 300:
        return "aktivitas belajar/kerja"
    if pir == 1 and ldr > 500:
        return "istirahat / minim aktivitas"
    if pir == 1 and 300 <= ldr <= 500:
        return "aktivitas normal"
    return "lain"

df["activity"] = df.apply(label_activity, axis=1)
df["hour"] = df["timestamp"].dt.hour

In [3]:

busy_hour_series = (
    df[df["activity"].str.contains("aktivitas", case=False)]
    ["hour"]
    .value_counts()
)
jam_paling_sibuk = busy_hour_series.idxmax()

rest_hour_series = (
    df[df["activity"].str.contains("istirahat", case=False)]
    ["hour"]
    .value_counts()
)

jam_istirahat = rest_hour_series.index.tolist()

empty_hour_series = (
    df[df["activity"] == "ruangan kosong"]
    ["hour"]
    .value_counts()
)

jam_paling_sepi = empty_hour_series.idxmax()

# print("\n=== Jam paling banyak aktivitas ===")
# print(busy_hours.head(3))

# print("\n=== Jam paling sering istirahat ===")
# print(rest_hours.head(3))

# print("\n=== Jam paling sering kosong ===")
# print(empty_hours.head(3))


df
# print(empty_hours)
# print(rest_hours)

def format_hour(h):
    return f"{h:02d}:00–{h:02d}:59"




In [4]:

print("\n=== Total Aktivitas ===")
print(df["activity"].value_counts())

print("\n=== Aktivitas per Jam ===")
print(df.groupby("hour")["activity"].agg(lambda x: x.value_counts().index[0]))

print("\n=== Rata-rata PIR per Jam ===")
print(df.groupby("hour")["pir"].mean())

print("\n=== Rata-rata LDR per Jam ===")
print(df.groupby("hour")["ldr"].mean())


=== Total Aktivitas ===
activity
ruangan kosong                 10323
orang baru keluar               1854
istirahat / minim aktivitas     1606
aktivitas belajar/kerja          376
lain                             133
aktivitas normal                  18
Name: count, dtype: int64

=== Aktivitas per Jam ===
hour
0     orang baru keluar
8        ruangan kosong
9        ruangan kosong
10       ruangan kosong
11       ruangan kosong
13       ruangan kosong
14       ruangan kosong
15       ruangan kosong
16       ruangan kosong
23    orang baru keluar
Name: activity, dtype: object

=== Rata-rata PIR per Jam ===
hour
0     0.184423
8     0.092777
9     0.138630
10    0.057971
11    0.073308
13    0.115977
14    0.127046
15    0.218650
16    0.160911
23    0.151961
Name: pir, dtype: float64

=== Rata-rata LDR per Jam ===
hour
0     361.530007
8     661.172185
9     691.595527
10    968.367150
11    778.781955
13    652.843471
14    723.808262
15    687.061093
16    680.119765
23    153.02941

In [ ]:
MQTT_HOST = ""
MQTT_PORT = 1883
MQTT_USER = ""
MQTT_PASS = ""
MQTT_TOPIC = "statistik/harian"

client = mqtt.Client()
client.username_pw_set(MQTT_USER, MQTT_PASS)
client.connect(MQTT_HOST, MQTT_PORT)

client.loop_start()   # 🔥 WAJIB

payload = {
    "ringkasan_harian": {
        "jam_paling_sibuk": format_hour(jam_paling_sibuk),
        "jam_paling_sepi": format_hour(jam_paling_sepi),
        "jam_istirahat": [format_hour(h) for h in jam_istirahat]
    }
}



result = client.publish(MQTT_TOPIC, json.dumps(payload))
result.wait_for_publish()

client.loop_stop()
client.disconnect()

print("Data statistik terkirim ke MQTT!")


Data statistik terkirim ke MQTT!


C:\Users\ASUS\AppData\Local\Temp\ipykernel_19500\3761468970.py:7: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()
